# OKX Data Collection and Strategy Backtesting

This notebook demonstrates how to:
1. Collect real market data from OKX exchange
2. Run a strategy backtest using the collected data
3. Analyze the results

## Setup

In [ ]:
import asyncio
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Import framework components
from src.data.collectors import OKXDataCollector
from src.strategies.implementations import CVDBBPullbackStrategy
from src.backtesting import VectorBTEngine
from src.analysis import PerformanceAnalyzer, RiskAnalyzer
from config.settings import settings

print("✅ All imports successful!")

## 1. Collect Data from OKX

Let's fetch recent BTC-USDT perpetual futures data:

In [ ]:
# Initialize data collector
collector = OKXDataCollector()

# Fetch recent data
symbol = "BTC-USDT-SWAP"
timeframe = "5m"
count = 50  # 5000 candles (about 17 days of 5-minute data)

print(f"🚀 Fetching {count * 100} candles for {symbol} ({timeframe})...")

# This is async, so we need to run it properly
df = await collector.fetch_historical_data(
    symbol=symbol,
    timeframe=timeframe, 
    count=count
)

print(f"✅ Successfully fetched {len(df)} candles")
print(f"📊 Data range: {df['datetime'].min()} to {df['datetime'].max()}")

# Display sample
df.head()

## 2. Visualize the Data

Let's create a candlestick chart to visualize the data:

In [ ]:
# Convert to pandas for plotting
pandas_df = collector.to_pandas(df)

# Create candlestick chart
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.1,
    subplot_titles=('Price', 'Volume'),
    row_width=[0.7, 0.3]
)

# Add candlestick
fig.add_trace(
    go.Candlestick(
        x=pandas_df['datetime'],
        open=pandas_df['open'],
        high=pandas_df['high'],
        low=pandas_df['low'],
        close=pandas_df['close'],
        name="Price"
    ),
    row=1, col=1
)

# Add volume
fig.add_trace(
    go.Bar(
        x=pandas_df['datetime'],
        y=pandas_df['volume'],
        name="Volume",
        marker_color='rgba(0,100,80,0.6)'
    ),
    row=2, col=1
)

fig.update_layout(
    title=f"{symbol} - {timeframe} Candlestick Chart",
    xaxis_rangeslider_visible=False,
    height=600
)

fig.show()

## 3. Prepare Data for Backtesting

Convert the data to the format needed for strategy backtesting:

In [ ]:
# Convert to pandas and set datetime index
backtest_data = pandas_df.copy()
backtest_data.set_index('datetime', inplace=True)

# Ensure we have all required columns
required_cols = ['open', 'high', 'low', 'close', 'volume']
missing_cols = [col for col in required_cols if col not in backtest_data.columns]

if missing_cols:
    print(f"❌ Missing columns: {missing_cols}")
else:
    print(f"✅ Data ready for backtesting with {len(backtest_data)} rows")
    print(f"📊 Columns: {list(backtest_data.columns)}")

backtest_data.tail()

## 4. Initialize and Run Strategy

Let's create and run the CVD Bollinger Band Pullback strategy:

In [ ]:
# Strategy parameters
strategy_params = {
    'bb_window': 20,
    'bb_std': 2.0,
    'rsi_window': 14,
    'rsi_overbought': 70,
    'rsi_oversold': 30,
    'volume_threshold': 1.5
}

# Initialize strategy
strategy = CVDBBPullbackStrategy(
    symbol=symbol,
    params=strategy_params
)

print(f"✅ Strategy initialized: {strategy.name}")
print(f"📋 Parameters: {strategy_params}")

In [ ]:
# Initialize backtesting engine
engine = VectorBTEngine()

# Run backtest
print("🚀 Running backtest...")

results = engine.run_backtest(
    strategy=strategy,
    data=backtest_data,
    initial_cash=10000,
    fees=0.001,  # 0.1% fees
    slippage=0.0005  # 0.05% slippage
)

print("✅ Backtest completed!")
print(f"📊 Portfolio value: ${results.portfolio.final_value:.2f}")
print(f"📈 Total return: {results.portfolio.total_return * 100:.2f}%")

## 5. Analyze Results

Let's analyze the backtest performance:

In [ ]:
# Initialize analyzers
perf_analyzer = PerformanceAnalyzer()
risk_analyzer = RiskAnalyzer()

# Calculate performance metrics
perf_metrics = perf_analyzer.calculate_metrics(
    portfolio=results.portfolio,
    benchmark_returns=None  # We could add a benchmark here
)

print("📊 Performance Metrics:")
print("=" * 40)
for metric, value in perf_metrics.items():
    if isinstance(value, float):
        if 'ratio' in metric.lower() or 'return' in metric.lower():
            print(f"{metric}: {value:.4f}")
        else:
            print(f"{metric}: {value:.2f}")
    else:
        print(f"{metric}: {value}")

In [ ]:
# Plot portfolio value over time
portfolio_value = results.portfolio.value

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=portfolio_value.index,
    y=portfolio_value.values,
    mode='lines',
    name='Portfolio Value',
    line=dict(color='green', width=2)
))

fig.update_layout(
    title="Portfolio Value Over Time",
    xaxis_title="Date",
    yaxis_title="Portfolio Value ($)",
    height=400
)

fig.show()

In [ ]:
# Plot trades on price chart
entries = results.portfolio.trades.entry_idx
exits = results.portfolio.trades.exit_idx

fig = go.Figure()

# Add price line
fig.add_trace(go.Scatter(
    x=backtest_data.index,
    y=backtest_data['close'],
    mode='lines',
    name='Price',
    line=dict(color='blue')
))

# Add entry points
if len(entries) > 0:
    entry_prices = backtest_data['close'].iloc[entries]
    fig.add_trace(go.Scatter(
        x=entry_prices.index,
        y=entry_prices.values,
        mode='markers',
        name='Entries',
        marker=dict(color='green', size=8, symbol='triangle-up')
    ))

# Add exit points
if len(exits) > 0:
    exit_prices = backtest_data['close'].iloc[exits]
    fig.add_trace(go.Scatter(
        x=exit_prices.index,
        y=exit_prices.values,
        mode='markers',
        name='Exits',
        marker=dict(color='red', size=8, symbol='triangle-down')
    ))

fig.update_layout(
    title="Strategy Trades on Price Chart",
    xaxis_title="Date",
    yaxis_title="Price",
    height=500
)

fig.show()

## 6. Risk Analysis

Let's analyze the risk characteristics:

In [ ]:
# Calculate risk metrics
returns = results.portfolio.returns
risk_metrics = risk_analyzer.calculate_risk_metrics(returns)

print("⚠️ Risk Metrics:")
print("=" * 40)
for metric, value in risk_metrics.items():
    if isinstance(value, float):
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

In [ ]:
# Plot returns distribution
returns_clean = returns.dropna()

fig = go.Figure()

fig.add_trace(go.Histogram(
    x=returns_clean * 100,  # Convert to percentage
    nbinsx=50,
    name='Returns Distribution',
    marker_color='lightblue',
    opacity=0.7
))

fig.update_layout(
    title="Returns Distribution",
    xaxis_title="Returns (%)",
    yaxis_title="Frequency",
    height=400
)

fig.show()

## Summary

🎉 **Congratulations!** You have successfully:

1. ✅ Collected real market data from OKX exchange
2. ✅ Visualized the price data
3. ✅ Ran a complete strategy backtest
4. ✅ Analyzed performance and risk metrics
5. ✅ Visualized trading signals and results

The framework is now ready for:
- Adding more data sources
- Implementing new strategies
- Running parameter optimization
- Performing walk-forward analysis
- Live trading deployment

Next steps:
1. Experiment with different strategy parameters
2. Add more symbols and timeframes
3. Implement additional strategies
4. Set up automated data collection
5. Deploy to live trading (when ready)